Решить три задачи из задачника. 
Найти приближенные решения, применяя метод монте-карло сравнить их с правильным ответом.

## § 2. Классическое вероятностное пространство

### Задача 2.49

**Условие**

Две игральные кости бросаются $r$ раз. Найти вероятность того, что каждая из шести комбинацияй $(1,1), ..., (6,6)$ появится по меньшей мере один раз.

**Аналитическое решение**

Пусть мы бросаем две честные игральные кости.  
Всего возможных результатов одного броска:

$$
36 = 6 \times 6
$$

Вероятность появления одной конкретной диагональной пары за один бросок:

$$
p = \frac{1}{36}
$$

Мы ищем вероятность того, что все шесть диагональных комбинаций  
$(1,1), (2,2), \dots, (6,6)$  
появятся хотя бы один раз за $r$ бросков.

Обозначим $A_i$ — событие «комбинация $(i,i)$ не выпала ни разу». 
Тогда нам нужна вероятность противоположного события:

$P(r) = 1 - \mathbb{P}(A_1 \cup A_2 \cup \dots \cup A_6)$.

Это *формула включений-исключений*. используем её, т.к. она позволяет точно вычислить вероятность события вида "все нужные события произошли хотя бы один раз" при условии, что эти события:
- зависят друг от друга;
- пересекаются;
- их невозможно посчитать напрямую через сумму.

Формула включений–исключений вычисляет вероятность того,  
что отсутствует хотя бы одна комбинация, учитывая все перекрытия таких случаев:

$P(r) =
\sum_{k=0}^{6}
(-1)^k \binom{6}{k}
\left(1 - \frac{k}{36}\right)^r$.

Как работает каждый элемент формулы:

- $\binom{6}{k}$ - число способов выбрать $k$ комбинаций из шести, которые могут не появиться.
- $(1 - k/36)^r$ - вероятность того, что выбранные $k$ комбинаций не появятся ни разу за $r$ бросков.
- $(-1)^k$ - корректирует пересечения: сначала добавляются одиночные случаи, затем вычитаются двойные пересечения, затем добавляются тройные и т.д.

Таким образом формула перебирает все варианты того, какие комбинации могли не выпасть,  
и в итоге даёт точную вероятность того, что не отсутствует ни одна комбинация,  
то есть все шесть диагональных пар появились хотя бы один раз.


**Python: аналитическое решение + Монте-Карло с разными генераторами случайных чисел**

In [14]:
import random
import math


h_line = ("-"*55)


# Аналитическое решение
def probability_analytical(n_rolls: int) -> float:
    """
    Аналитическая вероятность того, что все диагональные комбинации
    (1,1)...(6,6) появятся хотя бы раз за n_rolls бросков двух кубиков.

    Формула включений–исключений:
    P = Σ (-1)^k * C(6,k) * (1 - k/36)^n
    """
    total = 0.0
    for k in range(7):
        term = ((-1) ** k) * math.comb(6, k) * (1 - k / 36) ** n_rolls
        total += term
    return total


# Монте-Карло - единичный эксперемент
def run_single_experiment(n_rolls: int, rng: random.Random) -> bool:
    """
    True если ВСЕ диагональные пары (i,i) встретились хотя бы раз.
    """
    seen_diagonals = set()

    for _ in range(n_rolls):
        d1 = rng.randint(1, 6)
        d2 = rng.randint(1, 6)

        if d1 == d2:
            seen_diagonals.add(d1)

        if len(seen_diagonals) == 6:
            return True  # все 6 комбинаций собраны

    return len(seen_diagonals) == 6


# Монте-Карло
def probability_monte_carlo(n_rolls: int, n_experiments: int, seed: int = 42) -> float:
    rng = random.Random(seed)
    success_count = 0

    for _ in range(n_experiments):
        if run_single_experiment(n_rolls, rng):
            success_count += 1

    return success_count / n_experiments


# таблица
def print_results_table(n_rolls_list, n_experiments=50000):
    print(h_line)
    print(f"{'r (rolls)':>10} | {'Analytical':>12} | {'Monte Carlo':>12} | {'Delta':>10}")
    print(h_line)

    for n_rolls in n_rolls_list:
        p_an = probability_analytical(n_rolls)
        p_mc = probability_monte_carlo(n_rolls, n_experiments)
        delta = abs(p_an - p_mc)

        print(f"{n_rolls:10d} | {p_an:12.6f} | {p_mc:12.6f} | {delta:10.6f}")

    print(h_line)


if __name__ == "__main__":
    # кол-во бросков r
    r_values = [50, 100, 200, 300, 400, 500]
    
    # таблица
    print_results_table(r_values, n_experiments=50000)


-------------------------------------------------------
 r (rolls) |   Analytical |  Monte Carlo |      Delta
-------------------------------------------------------
        50 |     0.174088 |     0.172020 |   0.002068
       100 |     0.687507 |     0.688760 |   0.001253
       200 |     0.978720 |     0.979380 |   0.000660
       300 |     0.998719 |     0.998660 |   0.000059
       400 |     0.999923 |     0.999840 |   0.000083
       500 |     0.999995 |     1.000000 |   0.000005
-------------------------------------------------------


**Почему ошибка Монте-Карло уменьшается при увеличении числа бросков**

Оценка вероятности методом Монте-Карло - среднее значение бернуллиевской случайной величины:

$$
\hat{P} = \frac{1}{N} \sum_{i=1}^N I_i,
$$

где  

$$
I_i =
\begin{cases}
1, & \text{если в i-м эксперименте все 6 комбинаций } (k,k) \text{ появились хотя бы раз},\\[4pt]
0, & \text{иначе}.
\end{cases}
$$

Т.к. $\hat{P}$ — среднее из $N$ независимых бернуллиевских переменных, его дисперсия равна:

$$
\mathrm{Var}(\hat{P})
= \frac{P(r)\,\bigl(1 - P(r)\bigr)}{N}.
$$

**Что происходит при увеличении количества бросков $r$?**

Аналитическая вероятность стремится к 1:

$$
P(r) \rightarrow 1
\Rightarrow
1 - P(r) \rightarrow 0
\Rightarrow
\mathrm{Var}(\hat{P}) \rightarrow 0.
$$

Когда вероятность $P(r)$ близка к единице, почти каждый эксперимент Монте-Карло заканчивается "успехом".  
Колебания оценки становятся минимальными - метод Монте-Карло почти всегда выдаёт одно и то же значение.

Поэтому **разница между аналитикой и Монте-Карло резко уменьшается при больших $r$**.


### Задача 2.15

**Условие**

Колоду карт (36 листов) наудачу разделяют на 2 равные пачки. Чему равна вероятность, что:

а) в каждой из пачек окажется 2 туза;

б) в одной из пачек окажутся все 4 туза;

в) в пачках окажется по равному числу красных карт?

**Решение**


В задачах о разбиении колоды используется количество сочетаний:

$$
\binom{n}{k} = \frac{n!}{k!(n-k)!}
$$

Это число способов выбрать $k$ карт из $n$ без учёта порядка.

При разбиении 36-карточной колоды на две равные пачки по 18 карт общее число возможных разбиений:

$$
\text{Всего разбиений} = \binom{36}{18}.
$$

Во всех пунктах вероятность вычисляется как:

$$
P = \frac{\text{число благоприятных разбиений}}{\binom{36}{18}}.
$$

В Python эти сочетания вычисляются функцией:

```python
math.comb(n, k)
```

**(a) В каждой из пачек окажется 2 туза**

В колоде 4 туза. Чтобы в обеих пачках оказалось по 2 туза, в первую пачку должно попасть ровно 2 туза.

Число способов выбрать 2 туза из 4:

$$
\binom{4}{2}.
$$

Оставшиеся 16 карт добираются из 32 нетузов:

$$
\binom{32}{16}.
$$

Таким образом, число благоприятных разбиений:

$$
\binom{4}{2}\,\binom{32}{16}.
$$

Вероятность:

$$
P_a =
\frac{
\binom{4}{2}\,\binom{32}{16}
}{
\binom{36}{18}
}.
$$

In [19]:
import random
import math

# Аналитическая формула для пункта (а)
def prob_a_analytic():
    return (math.comb(4, 2) * math.comb(32, 16)) / math.comb(36, 18)

# Монте-Карло
def prob_a_mc(n_trials=100000, seed=42):
    rng = random.Random(seed)

    # строим колоду: 4 туза + 32 нетуза
    deck = ["A"] * 4 + ["N"] * 32

    success = 0

    for _ in range(n_trials):
        rng.shuffle(deck)
        pack1 = deck[:18]
        pack2 = deck[18:]

        if pack1.count("A") == 2 and pack2.count("A") == 2:
            success += 1

    return success / n_trials

print("Пункт (а)")
print("Аналитически:", prob_a_analytic())
print("Монте-Карло :", prob_a_mc())
print("Delta =", abs(prob_a_analytic() - prob_a_mc()))

Пункт (а)
Аналитически: 0.3974025974025974
Монте-Карло : 0.3953
Delta = 0.002102597402597439


**(б) в одной из пачек окажутся все 4 туза**

Для того чтобы все тузы оказались в первой пачке, нужно выбрать все 4 туза, а оставшиеся 14 карт добрать из 32 нетузов:

$$
\binom{32}{14}.
$$

То же количество разбиений соответствует случаю, когда все тузы во второй пачке.

Поэтому число благоприятных разбиений:

$$
2\,\binom{32}{14}.
$$

Вероятность:

$$
P_b =
\frac{
2\,\binom{32}{14}
}{
\binom{36}{18}
}.
$$

In [21]:
import random
import math

# Аналитическая формула для пункта (б)
def prob_b_analytic():
    return (2 * math.comb(32, 14)) / math.comb(36, 18)

# Монте-Карло
def prob_b_mc(n_trials=100000, seed=42):
    rng = random.Random(seed)

    deck = ["A"] * 4 + ["N"] * 32

    success = 0

    for _ in range(n_trials):
        rng.shuffle(deck)
        pack1 = deck[:18]
        pack2 = deck[18:]

        if pack1.count("A") == 4 or pack2.count("A") == 4:
            success += 1

    return success / n_trials

print("Пункт (б)")
print("Аналитически:", prob_b_analytic())
print("Монте-Карло :", prob_b_mc())
print("Delta =", abs(prob_b_analytic() - prob_b_mc()))

Пункт (б)
Аналитически: 0.1038961038961039
Монте-Карло : 0.10414
Delta = 0.0002438961038960935


**(в) в пачках окажется по равному числу красных карт**

В колоде 18 красных и 18 чёрных карт. Чтобы количество красных в обеих пачках совпадало, в первую пачку должно попасть ровно 9 красных.

Число способов выбрать 9 красных из 18:

$$
\binom{18}{9}.
$$

Аналогично выбираются 9 чёрных:

$$
\binom{18}{9}.
$$

Число благоприятных разбиений:

$$
\binom{18}{9}\,\binom{18}{9}.
$$

Вероятность:

$$
P_v =
\frac{
\binom{18}{9}\,\binom{18}{9}
}{
\binom{36}{18}
}.
$$

In [20]:
import random
import math

# Аналитическая формула для пункта (в)
def prob_v_analytic():
    return (math.comb(18, 9) * math.comb(18, 9)) / math.comb(36, 18)

# Монте-Карло
def prob_v_mc(n_trials=100000, seed=42):
    rng = random.Random(seed)

    # 18 красных, 18 чёрных
    deck = ["R"] * 18 + ["B"] * 18

    success = 0

    for _ in range(n_trials):
        rng.shuffle(deck)
        pack1 = deck[:18]
        pack2 = deck[18:]

        if pack1.count("R") == pack2.count("R"):
            success += 1

    return success / n_trials

print("Пункт (в)")
print("Аналитически:", prob_v_analytic())
print("Монте-Карло :", prob_v_mc())
print("Delta =", abs(prob_v_analytic() - prob_v_mc()))

Пункт (в)
Аналитически: 0.2604814497917183
Монте-Карло : 0.26126
Delta = 0.0007785502082817142
